**Bayesian Optimization for Evolutionary Scenarios**

Bayesian Optimization for all 9 selection scenarios

**Number of Trials: 50**

**Number of Simulations Used: 40,000**

**GPU Used: A100**

In [1]:

!git clone https://github.com/Djinho/EvoNet-CNN-Insight.git

# Navigate to the cloned repository
%cd EvoNet-CNN-Insight/Optimisation_notebooks

Cloning into 'EvoNet-CNN-Insight'...
remote: Enumerating objects: 6038, done.
remote: Counting objects: 100% (1659/1659), done.
remote: Compressing objects: 100% (1023/1023), done.
remote: Total 6038 (delta 711), reused 1525 (delta 618), pack-reused 4379
Receiving objects: 100% (6038/6038), 8.08 GiB | 17.50 MiB/s, done.
Resolving deltas: 100% (2302/2302), done.
Updating files: 100% (216/216), done.
/content/EvoNet-CNN-Insight/Optimisation_notebooks


In [2]:

!pip install optuna
!pip install optuna-integration

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.4/93.4 kB 9.0 MB/s eta 0:00:00


In [3]:
import optuna
import tensorflow as tf
from tensorflow.keras import models, layers, regularizers, callbacks
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import logging
import os
import time
import gzip
import skimage.transform
from optuna.integration import TFKerasPruningCallback

In [4]:

%run -i ../ImaGene.py


In [5]:
# Configure logging to file in the current directory with the specified name
logging.basicConfig(filename='Ancient_Scenarios_AM.log', level=logging.INFO,
                    format='%(asctime)s:%(levelname)s:%(message)s')

# Function to preprocess data
def preprocess_data(simulations_folder, batch_number):
    # Create ImaFile object for the specified simulation folder and model
    file_sim = ImaFile(simulations_folder=os.path.join(simulations_folder, f'Simulations{batch_number}'), nr_samples=198, model_name='Marth-3epoch-CEU')

    # Read the simulation data
    gene_sim = file_sim.read_simulations(parameter_name='selection_coeff_hetero', max_nrepl=4000)

    # Filter simulations based on frequency threshold
    gene_sim.filter_freq(0.01)

    # Sort simulations by row frequency
    gene_sim.sort('rows_freq')

    # Resize simulation data to the required dimensions
    gene_sim.resize((198, 192))

    # Convert simulation data (flip=True allows for data augmentation)
    gene_sim.convert(flip=True)

    # Get a random subset of indices and subset the simulation data
    gene_sim.subset(get_index_random(gene_sim))

    # Convert targets to binary format
    gene_sim.targets = to_binary(gene_sim.targets)

    return gene_sim

# Objective function for Optuna optimization
def objective(trial, simulations_folder, epochs=1):
    # Set random seed
    np.random.seed(42)
    tf.random.set_seed(42)

    # Hyperparameters to be optimized
    num_layers = trial.suggest_int('num_layers', 3, 6)  # Limit the number of layers to avoid excessive reduction
    filters = [trial.suggest_int(f'filters_{i}', 32, 128) for i in range(num_layers)]  # Adjust filters range
    kernel_size = trial.suggest_int('kernel_size', 2, 3)  # Limit the kernel size to avoid excessive reduction
    l1 = trial.suggest_float('l1', 1e-6, 1e-2, log=True)
    l2 = trial.suggest_float('l2', 1e-6, 1e-2, log=True)
    dense_units = trial.suggest_int('dense_units', 64, 256)  # Adjust dense units range
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-3, log=True)

    # Log the hyperparameters at the start of each trial
    logging.info(f'Trial {trial.number} started with hyperparameters: num_layers={num_layers}, filters={filters}, '
                 f'kernel_size={kernel_size}, l1={l1}, l2={l2}, dense_units={dense_units}, '
                 f'dropout_rate={dropout_rate}, learning_rate={learning_rate}')
    print(f'Trial {trial.number} started with hyperparameters: num_layers={num_layers}, filters={filters}, '
          f'kernel_size={kernel_size}, l1={l1}, l2={l2}, dense_units={dense_units}, '
          f'dropout_rate={dropout_rate}, learning_rate={learning_rate}')

    # Building the model based on sampled hyperparameters
    model = models.Sequential()
    model.add(layers.Input(shape=(198, 192, 1)))  # Define the input layer
    for i in range(num_layers):
        model.add(layers.Conv2D(filters=filters[i], kernel_size=(kernel_size, kernel_size), strides=(1, 1), activation='relu',
                                kernel_regularizer=regularizers.l1_l2(l1=l1, l2=l2), padding='same'))  # Use 'same' padding to avoid dimension issues
        model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(layers.Flatten())  # Flatten the output for the Dense layers
    model.add(layers.Dense(units=dense_units, activation='relu'))  # Add Dense layer
    model.add(layers.Dropout(rate=dropout_rate))  # Add Dropout layer
    model.add(layers.Dense(units=1, activation='sigmoid'))  # Output layer with sigmoid activation

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)  # Define optimizer
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])  # Compile the model

    # Pruning callback to terminate unpromising trials
    pruning_callback = TFKerasPruningCallback(trial, 'val_loss')

    # Early stopping to prevent overfitting
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    # Incremental training on 9 batches
    best_val_loss = np.inf  # Track the best validation loss
    for batch_number in range(1, 10):
        gene_sim = preprocess_data(simulations_folder, batch_number)  # Preprocess data
        history = model.fit(gene_sim.data, gene_sim.targets, batch_size=64, epochs=epochs, verbose=1, validation_split=0.2,
                            callbacks=[early_stopping, pruning_callback])  # Train the model with callbacks

        # Get the validation loss for the current epoch
        val_loss = min(history.history['val_loss'])
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            trial.report(val_loss, batch_number)
            if trial.should_prune():
                raise optuna.exceptions.TrialPruned()

    # Evaluate the model on the 10th batch
    gene_sim = preprocess_data(simulations_folder, 10)
    evaluation = model.evaluate(gene_sim.data, gene_sim.targets, verbose=1)
    return evaluation[1]  # Return the accuracy

# Function to run the optimization process
def run_optimization():
    start_time = time.time()  # Record start time

    simulations_folder = '/content/EvoNet-CNN-Insight/model_training/Ancient_moderate/AM'  # Folder for this dataset
    logging.info(f"Starting optimization for {simulations_folder}")
    print(f"Starting optimization for {simulations_folder}")

    # Create an Optuna study and optimize the objective function
    study = optuna.create_study(direction='maximize', pruner=optuna.pruners.MedianPruner(), storage='sqlite:///am_study.db', load_if_exists=True)  # Use SQLite storage
    study.optimize(lambda trial: objective(trial, simulations_folder), n_trials=50)  # Set to 50 trials

    try:
        trial = study.best_trial
        print('Best trial:')
        print('Value: {}'.format(trial.value))
        print('Params: ')
        for key, value in trial.params.items():
            print('    {}: {}'.format(key, value))  # Print best trial parameters

        logging.info('Best trial:')
        logging.info('Value: {}'.format(trial.value))
        logging.info('Params: ')
        for key, value in trial.params.items():
            logging.info('    {}: {}'.format(key, value))  # Log best trial parameters

    except ValueError as e:
        logging.error(f"No completed trials: {e}")  # Log error if no trials are completed

    elapsed_time = time.time() - start_time  # Calculate elapsed time
    print(f"Elapsed time: {elapsed_time:.2f} seconds")
    logging.info(f"Elapsed time: {elapsed_time:.2f} seconds")  # Log elapsed time

# Run the optimization process
if __name__ == '__main__':
    run_optimization()  # Execute the optimization function


Starting optimization for /content/EvoNet-CNN-Insight/model_training/Ancient_moderate/AM


[I 2024-08-11 21:04:32,025] A new study created in RDB with name: no-name-c492fb7f-dc7c-4412-9dbd-8d7338bfea6c


Trial 0 started with hyperparameters: num_layers=6, filters=[55, 60, 87, 50, 64, 33], kernel_size=2, l1=0.00035664681070305104, l2=0.0005729901632281326, dense_units=193, dropout_rate=0.3107556174793292, learning_rate=1.690623670619887e-05
50/50 ━━━━━━━━━━━━━━━━━━━━ 13s 79ms/step - accuracy: 0.4912 - loss: 2.2807 - val_accuracy: 0.5175 - val_loss: 2.2617
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.4865 - loss: 2.2570 - val_accuracy: 0.5200 - val_loss: 2.2383


/usr/local/lib/python3.10/dist-packages/optuna/trial/_trial.py:493: UserWarning: The reported value is ignored because this `step` 0 is already reported.
  warnings.warn(


50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.5119 - loss: 2.2325 - val_accuracy: 0.4725 - val_loss: 2.2162
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.5042 - loss: 2.2096 - val_accuracy: 0.5150 - val_loss: 2.1921
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.5149 - loss: 2.1866 - val_accuracy: 0.5225 - val_loss: 2.1693
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.5344 - loss: 2.1640 - val_accuracy: 0.5275 - val_loss: 2.1474
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.5222 - loss: 2.1422 - val_accuracy: 0.5962 - val_loss: 2.1251
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.5661 - loss: 2.1198 - val_accuracy: 0.5987 - val_loss: 2.1034
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.5483 - loss: 2.0986 - val_accuracy: 0.6137 - val_loss: 2.0821
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6110 - loss: 2.0821


...

Trial 48 started with hyperparameters: num_layers=3, filters=[40, 124, 103], kernel_size=3, l1=2.195655020140089e-06, l2=0.0006963405631647386, dense_units=239, dropout_rate=0.13236113984711123, learning_rate=1.4527206048438652e-05
49/50 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.5064 - loss: 0.8293

[I 2024-08-12 02:30:23,438] Trial 48 pruned. Trial was pruned at epoch 0.


Trial 49 started with hyperparameters: num_layers=4, filters=[97, 66, 43, 56], kernel_size=2, l1=1.045876041342991e-05, l2=0.002171021539639442, dense_units=210, dropout_rate=0.4140624319100813, learning_rate=3.339705968715223e-05
48/50 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.5085 - loss: 1.1111

[I 2024-08-12 02:31:55,453] Trial 49 pruned. Trial was pruned at epoch 0.


Best trial:
Value: 0.7225000262260437
Params: 
    num_layers: 5
    filters_0: 65
    filters_1: 128
    filters_2: 77
    filters_3: 48
    filters_4: 127
    kernel_size: 2
    l1: 5.39171365329603e-06
    l2: 0.000508098182990793
    dense_units: 122
    dropout_rate: 0.44775481260866545
    learning_rate: 0.00023021217142648836
Elapsed time: 19644.33 seconds
